# Utilisation du modèle SLM Phi4-mini dans Microsoft Fabric

In [1]:
#install onnx runtime
!pip install --pre onnxruntime-genai --q
import onnxruntime_genai as og
import os
import argparse
import time
import shutil

StatementMeta(, 843e3df9-e289-42b3-a616-fa69d91e7d6c, 3, Finished, Available, Finished)

In [2]:
model_path = '/lakehouse/default/Files/phi4slm'
if not os.path.exists(model_path) : os.mkdir(model_path)
print(model_path)

StatementMeta(, 843e3df9-e289-42b3-a616-fa69d91e7d6c, 4, Finished, Available, Finished)

/lakehouse/default/Files/phi4slm


In [4]:
!huggingface-cli download microsoft/Phi-4-mini-instruct-onnx --include cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/* --local-dir ./lakehouse/default/Files/phi4slm

StatementMeta(, 843e3df9-e289-42b3-a616-fa69d91e7d6c, 6, Finished, Available, Finished)

Fetching 11 files:   0%|                                 | 0/11 [00:00<?, ?it/s]Downloading 'cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/model.onnx' to 'lakehouse/default/Files/phi4slm/.huggingface/download/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/model.onnx.701aa5d185b6a782bc27104a990dd5b634fa507840b7c42f7ee6f1fb812d0b83.incomplete'

configuration_phi3.py: 100%|███████████████| 10.9k/10.9k [00:00<00:00, 55.6MB/s]
Download complete. Moving file to lakehouse/default/Files/phi4slm/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/configuration_phi3.py

merges.txt:   0%|                                   | 0.00/2.42M [00:00<?, ?B/s]Downloading 'cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/genai_config.json' to 'lakehouse/default/Files/phi4slm/.huggingface/download/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/genai_config.json.0fcfa1e663f2bc867f8dc62fae65dd0924f0a4d68b43d1234df742dd19171470.incomplete'


special_tokens_map.json: 100%|██████████████████| 587/587 [00:00<0

In [5]:
#copy files to a lakehouse from the temp directory
source_dir = os.path.abspath(".") + model_path
destination_dir = model_path

try:
    shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)
    print("Directory copied successfully.")
except Exception as e:
    print(f"Error occurred: {e}")

StatementMeta(, 843e3df9-e289-42b3-a616-fa69d91e7d6c, 7, Finished, Available, Finished)

Directory copied successfully.


In [20]:
import onnxruntime_genai as og

model = og.Model('/lakehouse/default/Files/phi4slm/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4')
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

# Set up search options
search_options = {"max_length": 500, "temperature": 0.6}
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

StatementMeta(, 843e3df9-e289-42b3-a616-fa69d91e7d6c, 22, Finished, Available, Finished)

In [21]:
prompt = "<|system|>You are a famous chess player.<|end|><|user|>Could you make a comparison between chess and football ? Imagine a storyline in french :)<|end|><|assistant|>"
generator = og.Generator(model, params)
generator.append_tokens(tokenizer.encode(prompt))
while not generator.is_done():
    generator.generate_next_token()
    [print(tokenizer_stream.decode(t), end='', flush=True) for t in generator.get_next_tokens()]
print()

StatementMeta(, 843e3df9-e289-42b3-a616-fa69d91e7d6c, 23, Finished, Available, Finished)

Certainement, voici une comparaison entre le jeu d'échecs et le football, présentée sous forme de récit en français :

Il était une fois, dans une petite ville pittoresque, deux passionnés, l'un un grand maître d'échecs, l'autre un footballeur talentueux. Ils se sont rencontrés par hasard lors d'un match de foot, et l'échange s'est transformé en une passionnante conversation sur leurs respectifs jeux stratégiques et tactiques.

Le grand maître d'échecs, Pierre, expliqua à son interlocuteur, Lucas, que l'échecs est un jeu d'esprit, où chaque joueur contrôle une armée infinie sur une grille d'échecs. La position initiale des pièces, l'ouverture, la défense, l'attaque et la conclusion sont toutes des moments cruciaux, où chaque décision peut influencer l'issue du match. La patience, la réflexion, la capacité à s'adapter et l'anticipation sont les traits de caractère essentiels pour réussir dans l'échecs.

Lucas, le footballeur, répondit que le football, bien que différent par sa dimension